# Machine Learning Using Pipeline

## Read and Split the Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('./tmp/train.csv', index_col='Id')
X_test_full = pd.read_csv('./tmp/test.csv', index_col='Id')

# Remove rows with missing target, esparate from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full, y, train_size=0.8, random_state=0)

## Find the Categorical and Numerical Features

In [2]:
# Categorical features
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique(
) < 10 and X_train_full[cname].dtype == 'object']

# Numerical features
numerical_cols = [
    cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [3]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

X_train.shape

(1168, 76)

In [4]:
X_valid.shape

(292, 76)

In [5]:
X_test.shape

(1459, 76)

## Pipeline

In [6]:
# improt packages
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#  Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])


## Define the model

In [7]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [8]:
# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

In [9]:
# Preprocessing of training data, fit model
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCo

In [10]:
# Prediction for valid data
preds = clf.predict(X_valid)

In [11]:
print("MAE", mean_absolute_error(y_valid, preds))

MAE 17621.3197260274


In [13]:
preds_test = clf.predict(X_test)

In [15]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('./tmp/submission.csv', index=False)